# Scraping Yahoo Finance

## Week 7. Practice Programming Assignment 1

In this assignment you are required to look at historical data for 30 companies from [Dow Jones Index](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average). Tickers for companies from the index can be found in *dow_jones_tickers.txt*. For each company you should get historical daily stock prices for 2019 from https://finance.yahoo.com/, and then use the data to answer the questions you will find below. 

###### Import coursera grader tools

In [1]:
import sys
sys.path.append("..")

<br><br><br><br>

### Coding part

In [ ]:
# you can start your web-scraping here

In [1]:
dow_jones_companies = []

with open('dow_jones_tickers.txt') as f:
    for ticker in f:
        dow_jones_companies.append(ticker.strip())

In [2]:
print(dow_jones_companies)

['AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'RTX', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [6]:
from bs4 import BeautifulSoup

In [300]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

ser = Service("C:/Users/mzaytsev/chromedriver")
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome(service=ser)

URL = "https://finance.yahoo.com/"
dow_URL = "https://finance.yahoo.com/quote/DOW/history?period1=1553040000&period2=1577836800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
# Empty list is created to store the data
extracted_data = []

for ticker in dow_jones_companies:
    # Hit the url of Yahoo Finance and wait for 2 seconds.
    if ticker == "DOW":
        driver.get(dow_URL)
    else:    
        driver.get(URL)

        search_box = WebDriverWait(driver, 180).until(
            EC.presence_of_element_located((By.ID,"yfin-usr-qry")))
        #search_box = driver.find_element(By.ID,"yfin-usr-qry")
        search_box.send_keys(ticker)
        time.sleep(6)
        stock_pg = search_box.send_keys(Keys.RETURN)
        time.sleep(6)

        hist_data = WebDriverWait(driver, 180).until(
            EC.element_to_be_clickable((By.XPATH,'//span[text()= "Historical Data"]')))
        #hist_data = driver.find_element(By.XPATH,'//span[text()= "Historical Data"]').click()
        hist_data.click()

        grab_time = WebDriverWait(driver, 180).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'div.dateRangeBtn')))
        #grab_time = driver.find_element(By.XPATH,'//span[text() = "Jul 16, 2021 - Jul 16, 2022"]').click()
        grab_time.click()

        start_time = WebDriverWait(driver, 180).until(
            EC.element_to_be_clickable((By.XPATH,'//input[@name = "startDate"]')))
        #start_time  = driver.find_element(By.XPATH,'//input[@name = "startDate"]')

        start_time.send_keys("01012019")

        end_time = WebDriverWait(driver, 180).until(
            EC.element_to_be_clickable((By.XPATH,'//input[@name = "endDate"]')))
        #end_time  = driver.find_element(By.XPATH,'//input[@name = "endDate"]')
        end_time.send_keys("01012020")


        clickDone = WebDriverWait(driver, 180).until(
            EC.element_to_be_clickable((By.XPATH,'//span[text() = "Done"]')))
        #clickDone = driver.find_element(By.XPATH,'//span[text() = "Done"]').click()
        clickDone.click()

        clickApply =  WebDriverWait(driver, 180).until(
            EC.element_to_be_clickable((By.XPATH,'//span[text() = "Apply"]')))
        #clickApply = driver.find_element(By.XPATH,'//span[text() = "Apply"]').click()
        clickApply.click()
    
    time.sleep(6)

    # Driver scrolls down three times to load the table.
    for i in range(0,3):
        driver.execute_script("window.scrollBy(0,5000)")
        time.sleep(3)

    HTMLPage = BeautifulSoup(driver.page_source, 'html.parser')

    # Table is searched using class and stored in another variable.
    Table = HTMLPage.find('table', class_='W(100%) M(0)')

    # List of all the rows is store in a variable 'Rows'.
    Rows = Table.find_all('tr', class_='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)')


    # Loop to go through each row of table
    for i in range(0, len(Rows)):
        try:
            # Empty dictionary to store data present in each row
            RowDict = {}
            # Extracted all the columns of a row and stored in a variable
            Values = Rows[i].find_all('td')

            # Values (Open, High, Close etc.) are extracted and stored in dictionary
            if len(Values) == 7:
                RowDict["Date"] = Values[0].find('span').text.replace(',', '')
                RowDict["Ticker"] = ticker
                RowDict["Open"] = Values[1].find('span').text.replace(',', '')
                RowDict["High"] = Values[2].find('span').text.replace(',', '')
                RowDict["Low"] = Values[3].find('span').text.replace(',', '')
                RowDict["Close"] = Values[4].find('span').text.replace(',', '')
                RowDict["Adj Close"] = Values[5].find('span').text.replace(',', '')
                RowDict["Volume"] = Values[6].find('span').text.replace(',', '')
                # Uncomment below print statement if required
                # print(RowDict) 
                # Dictionary is appended in list
                extracted_data.append(RowDict)
        except:
            # To check the exception caused for which company
            print("Row Number: " + str(i))
        finally:
            # To move to the next row
            i = i + 1

# Converted list of dictionaries to a Dataframe.
extracted_data = pd.DataFrame(extracted_data)
print(extracted_data)
#driver.close()

             Date Ticker    Open    High     Low   Close Adj Close   Volume
0     Dec 31 2019    AXP  124.29  124.57  123.78  124.49    119.80  2340400
1     Dec 30 2019    AXP  125.20  125.46  124.18  124.30    119.62  2306500
2     Dec 27 2019    AXP  125.84  125.97  125.11  125.19    120.48  1788600
3     Dec 26 2019    AXP  124.98  125.44  124.53  125.41    120.69  1486600
4     Dec 24 2019    AXP  124.95  125.33  124.38  124.74    120.05   953500
...           ...    ...     ...     ...     ...     ...       ...      ...
7502  Mar 26 2019    DOW   49.00   49.75   48.18   48.85     41.12   504700
7503  Mar 25 2019    DOW   48.60   49.40   48.00   49.15     41.37   440900
7504  Mar 22 2019    DOW   48.80   49.95   48.16   48.60     40.91   844700
7505  Mar 21 2019    DOW   49.99   50.00   48.20   48.98     41.23  1764700
7506  Mar 20 2019    DOW   52.75   53.50   49.50   49.80     41.92  2350800

[7507 rows x 8 columns]


In [301]:
extracted_data.to_csv("yahoo_tickers.csv")

In [302]:
columns = extracted_data.columns
print(columns[2:-1])
extracted_data[columns[2:-1]] = extracted_data[columns[2:-1]].apply(pd.to_numeric)
extracted_data["Volume"] = extracted_data["Volume"].astype(int)
extracted_data.dtypes

Index(['Open', 'High', 'Low', 'Close', 'Adj Close'], dtype='object')


Date          object
Ticker        object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int32
dtype: object

In [303]:
extracted_data.head()

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,Dec 31 2019,AXP,124.29,124.57,123.78,124.49,119.80,2340400
1,Dec 30 2019,AXP,125.20,125.46,124.18,124.30,119.62,2306500
2,Dec 27 2019,AXP,125.84,125.97,125.11,125.19,120.48,1788600
3,Dec 26 2019,AXP,124.98,125.44,124.53,125.41,120.69,1486600
4,Dec 24 2019,AXP,124.95,125.33,124.38,124.74,120.05,953500


In [304]:
extracted_data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,7507.000000,7507.000000,7507.000000,7507.000000,7507.000000,7.507000e+03
mean,122.856579,123.831818,121.843762,122.892026,114.805605,1.220437e+07
std,71.268443,71.943117,70.582645,71.308764,69.680761,2.169640e+07
min,32.540000,32.810000,32.230000,32.490000,29.280000,4.409000e+05
25%,63.535000,64.045000,62.580000,63.445000,57.105000,3.659050e+06
50%,116.400000,117.230000,115.550000,116.380000,106.710000,6.229700e+06
75%,146.475000,147.470000,145.455000,146.430000,139.095000,1.198560e+07
max,446.010000,446.010000,440.190000,440.620000,430.300000,3.652488e+08


In [305]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7507 entries, 0 to 7506
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       7507 non-null   object 
 1   Ticker     7507 non-null   object 
 2   Open       7507 non-null   float64
 3   High       7507 non-null   float64
 4   Low        7507 non-null   float64
 5   Close      7507 non-null   float64
 6   Adj Close  7507 non-null   float64
 7   Volume     7507 non-null   int32  
dtypes: float64(5), int32(1), object(2)
memory usage: 440.0+ KB


<br><br>

### Questions

<br><br>

**Question 1.** What is the average change of price over the year (in %)?

*Note 1*. The opening price is the price at which a stock first trades upon the opening of an exchange on a trading day.

*Note 2*. The closing price for any stock is the final price at which it trades during regular market hours on any given day.

*Note 3*. Here by the price change we going to mean a ratio of a closing price in the last day of the period to an opening price in the first day of that period, subtracted one and multiplied by 100.

Example: if a price of a stock in day 1 opened at \\$100, and its close price in the last day was \\$120, then the price change during the period is: $$\left(\dfrac{120}{100}-1\right) * 100 = (1.2 - 1) * 100=20.$$

The price grew by 20%.

In [306]:
extracted_data = pd.read_csv("yahoo_tickers.csv")
extracted_data = extracted_data.reindex(index=extracted_data.index[::-1]).reset_index()
extracted_data = extracted_data.drop("index", axis=1)
extracted_data = extracted_data.drop("Unnamed: 0", axis=1)
extracted_data["Date"] = pd.to_datetime(extracted_data["Date"], format='%b %d %Y')
extracted_data = extracted_data.sort_values(["Ticker", "Date"])
extracted_data["change_of_price"] = (extracted_data["Close"]/extracted_data["Open"] - 1)*100
extracted_data.head()

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
7003,2019-01-02,AAPL,38.72,39.71,38.56,39.48,38.22,148158800,1.962810
7004,2019-01-03,AAPL,35.99,36.43,35.50,35.55,34.41,365248800,-1.222562
7005,2019-01-04,AAPL,36.13,37.14,35.95,37.06,35.88,234428400,2.574038
7006,2019-01-07,AAPL,37.17,37.21,36.47,36.98,35.80,219111200,-0.511165
7007,2019-01-08,AAPL,37.39,37.96,37.13,37.69,36.49,164101200,0.802354


In [307]:
extracted_data.shape

(7507, 9)

In [308]:
extracted_data.groupby("Ticker")["Date"].count()

Ticker
AAPL    252
AXP     252
BA      252
CAT     252
CSCO    252
CVX     252
DIS     252
DOW     199
GS      252
HD      252
IBM     252
INTC    252
JNJ     252
JPM     252
KO      252
MCD     252
MMM     252
MRK     252
MSFT    252
NKE     252
PFE     252
PG      252
RTX     252
TRV     252
UNH     252
V       252
VZ      252
WBA     252
WMT     252
XOM     252
Name: Date, dtype: int64

In [309]:
grouped = extracted_data.sort_values(["Ticker", "Date"]).groupby(["Ticker"]).agg(Open_f= ('Open', lambda x: list(x)[0]), Close_l =('Close', lambda x: list(x)[-1]))
grouped["change_of_price"] = (grouped["Close_l"]/grouped["Open_f"] - 1)*100
grouped.head(100)

,Open_f,Close_l,change_of_price
Ticker,,,
AAPL,38.72,73.41,89.591942
AXP,93.91,124.49,32.563092
BA,316.19,325.76,3.026661
CAT,124.03,147.68,19.067967
CSCO,42.28,47.96,13.434248
CVX,107.34,120.51,12.269424
DIS,108.10,144.63,33.792784
DOW,52.75,54.73,3.753555
GS,164.33,229.93,39.919674


In [310]:
grouped.iloc[0].values

array([38.72      , 73.41      , 89.59194215])

In [311]:
grouped["change_of_price"].mean()

23.665016202328882

In [253]:
import os
cwd = os.getcwd()
os.path.join( cwd, "Data", ticker,".csv" )
print(cwd)

C:\Users\mzaytsev\PycharmProjects\hse-coursera-data-scraping\week7


In [290]:
mean_change = []
for ticker in  dow_jones_companies:
    path = os.path.join( cwd, "Data", ticker )
    df = pd.read_csv(path+".csv")
    mean_change.append((df.iloc[-1].Close/df.iloc[0].Open -1)*100)
    print(ticker,round(df.iloc[0].Open,2),grouped.loc[ticker]["Open_f"],round(df.iloc[-1].Close,2),grouped.loc[ticker]["Close_l"],df.iloc[-1]["Date"],df.iloc[0]["Date"])
print(sum(mean_change)/len(mean_change))

AXP 93.91 93.91 124.49 124.49 2019-12-31 2019-01-02
AAPL 38.72 38.72 73.41 73.41 2019-12-31 2019-01-02
BA 316.19 316.19 325.76 325.76 2019-12-31 2019-01-02
CAT 124.03 124.03 147.68 147.68 2019-12-31 2019-01-02
CSCO 42.28 42.28 47.96 47.96 2019-12-31 2019-01-02
CVX 107.34 107.34 120.51 120.51 2019-12-31 2019-01-02
XOM 67.35 67.35 69.78 69.78 2019-12-31 2019-01-02
GS 164.33 164.33 229.93 229.93 2019-12-31 2019-01-02
HD 169.71 169.71 218.38 218.38 2019-12-31 2019-01-02
IBM 107.08 107.08 128.15 128.15 2019-12-31 2019-01-02
INTC 45.96 45.96 59.85 59.85 2019-12-31 2019-01-02
JNJ 128.13 128.13 145.87 145.87 2019-12-31 2019-01-02
KO 46.94 46.94 55.35 55.35 2019-12-31 2019-01-02
JPM 95.95 95.95 139.4 139.4 2019-12-31 2019-01-02
MCD 175.41 175.41 197.61 197.61 2019-12-31 2019-01-02
MMM 187.82 187.82 176.42 176.42 2019-12-31 2019-01-02
MRK 71.84 71.84 86.78 86.78 2019-12-31 2019-01-02
MSFT 99.55 99.55 157.7 157.7 2019-12-31 2019-01-02
NKE 72.79 72.79 101.31 101.31 2019-12-31 2019-01-02
PFE 40.91 

In [207]:
answer_part_1 = 23.474980968251415

<br>

**Question 2.** What company's stock price grew the most (in %)? Enter ticker of the company as an answer)

In [208]:
grouped[grouped["change_of_price"]==grouped["change_of_price"].max()]

,Open_f,Close_l,change_of_price
Ticker,,,
AAPL,38.72,72.88,88.22314


In [209]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].max()]

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
6769,2019-03-11,BA,371.27,402.67,365.55,400.01,390.64,34742200,7.740997


In [210]:
answer_part_2 = "WBA"

<br>

**Question 3.** What company's stock lost in price the most (in %)? Enter ticker of the company as an answer

In [211]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].min()]

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
1775,2019-04-16,UNH,238.01,238.01,215.82,220.96,210.23,27361400,-7.163565


In [212]:
grouped[grouped["change_of_price"]==grouped["change_of_price"].min()]

,Open_f,Close_l,change_of_price
Ticker,,,
WBA,67.2,58.91,-12.33631


In [213]:
answer_part_3 = "WBA"

<br>

**Question 4.** What company had the largest summary volume over the year? Enter ticker of the company as an answer

In [214]:
volume = extracted_data.groupby(["Ticker"])["Volume"].sum()
#volume[volume["Volume"]==volume["Volume"].max()]
volume.idxmax()

'AAPL'

In [215]:
answer_part_4 = 'AAPL'

<br>

**Question 5.** What is the biggest stock price daily increase (in %)? Enter the number 

In [216]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].max()]

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
6769,2019-03-11,BA,371.27,402.67,365.55,400.01,390.64,34742200,7.740997


In [217]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].max()]["change_of_price"].item()

7.740997118000381

In [218]:
answer_part_5 = 7.740997118000381

<br><br>

**Question 6.** What is the company that had the biggest stock price daily increase? Enter ticker of the company as an answer

In [219]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].max()]

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
6769,2019-03-11,BA,371.27,402.67,365.55,400.01,390.64,34742200,7.740997


In [220]:
answer_part_6 = "BA"

<br>

**Question 7.** What is the biggest stock price daily decrease (in %)? Enter the number

In [221]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].min()]

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
1775,2019-04-16,UNH,238.01,238.01,215.82,220.96,210.23,27361400,-7.163565


In [222]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].min()]["change_of_price"].item()

-7.16356455611108

In [223]:
answer_part_7 = ...

<br>

**Question 8.** What is the company that had the biggest stock price daily decrese (in %)? Enter ticker of the company as an answer

In [224]:
extracted_data[extracted_data["change_of_price"]==extracted_data["change_of_price"].min()]

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
1775,2019-04-16,UNH,238.01,238.01,215.82,220.96,210.23,27361400,-7.163565


In [225]:
answer_part_8 = "UNH"

<br>

**Question 9.** What was the best month for all companies (i.e. average monthly price increase was the best)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [226]:
extracted_data[extracted_data["Ticker"]=="DOW"]

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,change_of_price
0,2019-03-21,DOW,49.99,50.00,48.20,48.98,41.23,1764700,-2.020404
1,2019-03-22,DOW,48.80,49.95,48.16,48.60,40.91,844700,-0.409836
2,2019-03-25,DOW,48.60,49.40,48.00,49.15,41.37,440900,1.131687
3,2019-03-26,DOW,49.00,49.75,48.18,48.85,41.12,504700,-0.306122
4,2019-03-27,DOW,49.00,50.78,49.00,50.10,42.17,1788600,2.244898
...,...,...,...,...,...,...,...,...,...
192,2019-12-23,DOW,54.49,55.05,54.22,54.93,48.31,2678600,0.807488
193,2019-12-24,DOW,55.00,55.25,54.65,54.90,48.28,926200,-0.181818
194,2019-12-26,DOW,54.98,55.23,54.52,55.22,48.57,1909600,0.436522
195,2019-12-27,DOW,55.29,55.55,54.72,54.83,48.22,1688500,-0.831977


In [235]:
extracted_data["Month"] = extracted_data["Date"].dt.month_name()
grouped_m = extracted_data.groupby(["Ticker","Month"]).agg(Open_f= ('Open', lambda x: list(x)[0]), Close_l =('Close', lambda x: list(x)[-1]))
grouped_m["change_of_price"] = (grouped_m["Close_l"]/grouped_m["Open_f"] - 1)*100
grouped_m = grouped_m.groupby("Month")["change_of_price"].mean()
grouped_m

Month
April        1.984414
August      -2.751223
December     1.802325
February     3.382967
January      7.409273
July        -0.240822
June         7.062817
March        0.483739
May         -6.664097
November     2.573628
October      0.697505
September    3.443022
Name: change_of_price, dtype: float64

In [236]:
grouped_m.idxmax()

'January'

In [237]:
answer_part_9 = ...

<br>

**Question 10.** What was the worst month for all companies (i.e. average monthly price increase was the worst)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [239]:
grouped_m.idxmin()

'May'

In [ ]:
answer_part_10 = ...

In [ ]:
# Setting our answers to grader. Do not change!
grader.set_answer("oD7pP", answer_part_10)

<br>
<br>

### Submitting answers

In [ ]:
# you can make submission with answers so far to check yourself at this stage
grader.submit(COURSERA_EMAIL, COURSERA_TOKEN)